In [2]:
%%capture
!pip install transformers==4.31 # version specific 

In [1]:
from lavis.models import model_zoo

/opt/conda/envs/pytorch/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


In [8]:
!ls lavis/models

__init__.py   base_model.py	     clip_models  img2prompt_models  vit.py
__pycache__   blip2_models	     clip_vit.py  med.py
albef_models  blip_diffusion_models  eva_vit.py   pnp_vqa_models
alpro_models  blip_models	     gpt_models   timesformer


In [9]:
import torch
from torch import nn
from torchvision import transforms

from PIL import Image
import requests

from lavis.models import load_model_and_preprocess, load_model

from functools import partial
from lavis.models import VisionTransformer
from lavis.models.xbert import BertConfig, BertModel
from lavis.models.tokenization_bert import BertTokenizer

ImportError: cannot import name 'VisionTransformer' from 'lavis.models' (/home/ubuntu/LAVIS_BLIP/lavis/models/__init__.py)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### BLIP Example

In [ ]:
class VL_Transformer_ITM(nn.Module):
    def __init__(self,                 
                 text_encoder = None,
                 config_bert = ''
                 ):
        super().__init__()
    
        bert_config = BertConfig.from_json_file(config_bert)

        self.visual_encoder = VisionTransformer(
            img_size=384, patch_size=16, embed_dim=768, depth=12, num_heads=12, 
            mlp_ratio=4, qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6)) 

        self.text_encoder = BertModel.from_pretrained(text_encoder, config=bert_config, add_pooling_layer=False)   
        
        self.itm_head = nn.Linear(768, 2) 

        
    def forward(self, image, text):
        image_embeds = self.visual_encoder(image) 

        image_atts = torch.ones(image_embeds.size()[:-1],dtype=torch.long).to(image.device)

        output = self.text_encoder(text.input_ids, 
                                attention_mask = text.attention_mask,
                                encoder_hidden_states = image_embeds,
                                encoder_attention_mask = image_atts,      
                                return_dict = True,
                               )     
           
        vl_embeddings = output.last_hidden_state[:,0,:]
        vl_output = self.itm_head(vl_embeddings)   
        return vl_output